In [1]:
# =================================EL 1: DEFINISI =================================
# (Jalankan sel ini terlebih dahulu untuk membuat definisi tersedia)

import tkinter as tk
from tkinter import filedialog, messagebox, scrolledtext
import secrets # Untuk pembuatan kunci yang aman secara kriptografis
import os # Untuk mendapatkan nama file dasar

# --- Operasi Inti OTP ---

def generate_otp_key_internal(length_bytes):
    """
    Menghasilkan kunci OTP acak (dalam bentuk byte) dengan panjang tertentu.
    Fungsi internal, validasi panjang dilakukan sebelum memanggil ini.
    """
    return secrets.token_bytes(length_bytes)

def xor_bytes_operation(data_bytes, key_bytes):
    """
    Melakukan operasi XOR antara dua byte string.
    Memastikan panjang data dan kunci sama sebelum operasi.
    """
    if len(data_bytes) != len(key_bytes):
        # Ini seharusnya tidak terjadi jika logika aplikasi benar
        raise ValueError("Panjang data dan kunci harus sama untuk operasi XOR.")
    return bytes([a ^ b for a, b in zip(data_bytes, key_bytes)])

# --- Kelas Aplikasi Utama OTP ---

class OTPApplicationGUI:
    def __init__(self, master_root):
        self.master_root = master_root
        self.master_root.title("Program Enkripsi OTP Sederhana")
        self.master_root.geometry("750x700") 

        self.current_active_key_bytes = None 

        self.master_root.columnconfigure(1, weight=1)
        for i in range(9):
            self.master_root.rowconfigure(i, pad=7)

        tk.Label(master_root, text="Pesan (Plaintext / Ciphertext Hex):").grid(row=0, column=0, padx=10, pady=5, sticky="w")
        self.message_input_text = scrolledtext.ScrolledText(master_root, wrap=tk.WORD, height=8, width=80)
        self.message_input_text.grid(row=1, column=0, columnspan=2, padx=10, pady=5, sticky="ew")

        tk.Label(master_root, text="Kunci OTP (Format Hexadecimal):").grid(row=2, column=0, padx=10, pady=5, sticky="w")
        self.key_display_text = scrolledtext.ScrolledText(master_root, wrap=tk.WORD, height=5, width=80)
        self.key_display_text.grid(row=3, column=0, columnspan=2, padx=10, pady=5, sticky="ew")
        self.key_display_text.config(state=tk.DISABLED) 

        key_buttons_frame = tk.Frame(master_root)
        key_buttons_frame.grid(row=4, column=0, columnspan=2, pady=5, sticky="w", padx=10)

        self.generate_key_button = tk.Button(key_buttons_frame, text="Buat Kunci Baru (dari Pesan)", command=self.action_generate_and_display_key)
        self.generate_key_button.pack(side=tk.LEFT, padx=5)

        self.load_key_button = tk.Button(key_buttons_frame, text="Muat Kunci dari File", command=self.action_load_key_from_file)
        self.load_key_button.pack(side=tk.LEFT, padx=5)

        self.save_key_button = tk.Button(key_buttons_frame, text="Simpan Kunci ke File", command=self.action_save_key_to_file)
        self.save_key_button.pack(side=tk.LEFT, padx=5)
        self.save_key_button.config(state=tk.DISABLED) 

        action_buttons_frame = tk.Frame(master_root)
        action_buttons_frame.grid(row=5, column=0, columnspan=2, pady=10, sticky="w", padx=10)

        self.encrypt_button = tk.Button(action_buttons_frame, text="Enkripsi Pesan", command=self.action_encrypt_message)
        self.encrypt_button.pack(side=tk.LEFT, padx=5)

        self.decrypt_button = tk.Button(action_buttons_frame, text="Dekripsi Pesan", command=self.action_decrypt_message)
        self.decrypt_button.pack(side=tk.LEFT, padx=5)

        tk.Label(master_root, text="Hasil Operasi:").grid(row=6, column=0, padx=10, pady=5, sticky="w")
        self.result_output_text = scrolledtext.ScrolledText(master_root, wrap=tk.WORD, height=8, width=80)
        self.result_output_text.grid(row=7, column=0, columnspan=2, padx=10, pady=5, sticky="ew")
        self.result_output_text.config(state=tk.DISABLED) 

        self.status_bar_label = tk.Label(master_root, text="Status: Siap", bd=1, relief=tk.SUNKEN, anchor=tk.W)
        self.status_bar_label.grid(row=8, column=0, columnspan=2, sticky="ew", padx=10, pady=5)

    def _update_gui_status(self, message_text):
        self.status_bar_label.config(text=f"Status: {message_text}")

    def action_generate_and_display_key(self):
        plaintext_string = self.message_input_text.get("1.0", tk.END).strip()
        if not plaintext_string:
            messagebox.showerror("Kesalahan Input", "Masukkan pesan terlebih dahulu untuk menentukan panjang kunci.")
            self._update_gui_status("Gagal buat kunci: Pesan input kosong.")
            return
        try:
            plaintext_bytes_for_length = plaintext_string.encode('utf-8')
        except UnicodeEncodeError:
            messagebox.showerror("Kesalahan Encoding", "Pesan mengandung karakter yang tidak dapat di-encode ke UTF-8.")
            self._update_gui_status("Gagal buat kunci: Error encoding pesan.")
            return
        if len(plaintext_bytes_for_length) == 0:
            messagebox.showerror("Kesalahan Input", "Pesan tidak boleh kosong untuk pembuatan kunci.")
            self._update_gui_status("Gagal buat kunci: Pesan menghasilkan 0 byte.")
            return
        self.current_active_key_bytes = generate_otp_key_internal(len(plaintext_bytes_for_length))
        self.key_display_text.config(state=tk.NORMAL)
        self.key_display_text.delete("1.0", tk.END)
        self.key_display_text.insert(tk.END, self.current_active_key_bytes.hex())
        self.key_display_text.config(state=tk.DISABLED)
        self.save_key_button.config(state=tk.NORMAL)
        self._update_gui_status(f"Kunci baru ({len(self.current_active_key_bytes)} bytes) berhasil dibuat.")

    def action_load_key_from_file(self):
        file_path_selected = filedialog.askopenfilename(
            title="Pilih File Kunci OTP",
            filetypes=(("File Kunci OTP", "*.key"), ("Semua File", "*.*"))
        )
        if not file_path_selected:
            self._update_gui_status("Pemuatan kunci dibatalkan oleh pengguna.")
            return
        try:
            with open(file_path_selected, 'rb') as key_file:
                loaded_key_bytes = key_file.read()
            if not loaded_key_bytes:
                messagebox.showerror("Kesalahan File Kunci", "File kunci yang dipilih kosong atau tidak valid.")
                self.current_active_key_bytes = None
                self.key_display_text.config(state=tk.NORMAL)
                self.key_display_text.delete("1.0", tk.END)
                self.key_display_text.config(state=tk.DISABLED)
                self.save_key_button.config(state=tk.DISABLED)
                self._update_gui_status("Gagal muat kunci: File kosong.")
                return
            self.current_active_key_bytes = loaded_key_bytes
            self.key_display_text.config(state=tk.NORMAL)
            self.key_display_text.delete("1.0", tk.END)
            self.key_display_text.insert(tk.END, self.current_active_key_bytes.hex())
            self.key_display_text.config(state=tk.DISABLED)
            self.save_key_button.config(state=tk.NORMAL)
            self._update_gui_status(f"Kunci ({len(self.current_active_key_bytes)} bytes) berhasil dimuat dari: {os.path.basename(file_path_selected)}")
        except Exception as e:
            messagebox.showerror("Kesalahan Membaca File", f"Gagal memuat kunci dari file: {e}")
            self.current_active_key_bytes = None
            # (Reset key display and save button state as in the original code)
            self._update_gui_status(f"Gagal muat kunci: {e}")


    def action_save_key_to_file(self):
        if not self.current_active_key_bytes:
            messagebox.showerror("Kesalahan Penyimpanan", "Tidak ada kunci aktif untuk disimpan.")
            self._update_gui_status("Gagal simpan kunci: Kunci tidak ada.")
            return
        file_path_to_save = filedialog.asksaveasfilename(
            title="Simpan File Kunci OTP",
            defaultextension=".key",
            filetypes=(("File Kunci OTP", "*.key"), ("Semua File", "*.*"))
        )
        if not file_path_to_save:
            self._update_gui_status("Penyimpanan kunci dibatalkan oleh pengguna.")
            return
        try:
            with open(file_path_to_save, 'wb') as key_file:
                key_file.write(self.current_active_key_bytes)
            self._update_gui_status(f"Kunci berhasil disimpan ke: {os.path.basename(file_path_to_save)}")
        except Exception as e:
            messagebox.showerror("Kesalahan Menyimpan File", f"Gagal menyimpan kunci ke file: {e}")
            self._update_gui_status(f"Gagal simpan kunci: {e}")

    def _get_message_bytes_from_input(self, for_encryption_flag=True):
        text_string_input = self.message_input_text.get("1.0", tk.END).strip()
        if not text_string_input:
            messagebox.showerror("Kesalahan Input", "Area pesan tidak boleh kosong untuk operasi ini.")
            return None
        try:
            if for_encryption_flag:
                return text_string_input.encode('utf-8')
            else: 
                return bytes.fromhex(text_string_input)
        except ValueError: 
             messagebox.showerror("Kesalahan Format Ciphertext", "Format ciphertext tidak valid. Harap masukkan string hexadecimal yang benar.")
             return None
        except UnicodeEncodeError:
            messagebox.showerror("Kesalahan Encoding Pesan", "Pesan mengandung karakter yang tidak dapat di-encode ke UTF-8.")
            return None

    def action_encrypt_message(self):
        plaintext_bytes_to_encrypt = self._get_message_bytes_from_input(for_encryption_flag=True)
        if not plaintext_bytes_to_encrypt:
            self._update_gui_status("Enkripsi gagal: Pesan input tidak valid.")
            return
        if not self.current_active_key_bytes:
            messagebox.showerror("Kesalahan Kunci", "Kunci OTP belum ada (buat atau muat kunci terlebih dahulu).")
            self._update_gui_status("Enkripsi gagal: Kunci tidak tersedia.")
            return
        if len(plaintext_bytes_to_encrypt) != len(self.current_active_key_bytes):
            messagebox.showerror("Kesalahan Panjang Data", f"Panjang pesan ({len(plaintext_bytes_to_encrypt)} bytes) dan kunci ({len(self.current_active_key_bytes)} bytes) harus sama.")
            self._update_gui_status("Enkripsi gagal: Panjang pesan dan kunci tidak cocok.")
            return
        try:
            ciphertext_result_bytes = xor_bytes_operation(plaintext_bytes_to_encrypt, self.current_active_key_bytes)
        except ValueError as e:
            messagebox.showerror("Kesalahan Operasi", f"Terjadi kesalahan saat enkripsi: {e}")
            self._update_gui_status(f"Enkripsi gagal: {e}")
            return
        self.result_output_text.config(state=tk.NORMAL)
        self.result_output_text.delete("1.0", tk.END)
        self.result_output_text.insert(tk.END, ciphertext_result_bytes.hex())
        self.result_output_text.config(state=tk.DISABLED)
        self._update_gui_status("Pesan berhasil dienkripsi. Hasil (ciphertext) ditampilkan dalam format Hex.")

    def action_decrypt_message(self):
        ciphertext_bytes_to_decrypt = self._get_message_bytes_from_input(for_encryption_flag=False)
        if not ciphertext_bytes_to_decrypt:
            self._update_gui_status("Dekripsi gagal: Ciphertext input tidak valid.")
            return
        if not self.current_active_key_bytes:
            messagebox.showerror("Kesalahan Kunci", "Kunci OTP belum ada (buat atau muat kunci terlebih dahulu).")
            self._update_gui_status("Dekripsi gagal: Kunci tidak tersedia.")
            return
        if len(ciphertext_bytes_to_decrypt) != len(self.current_active_key_bytes):
            messagebox.showerror("Kesalahan Panjang Data", f"Panjang ciphertext ({len(ciphertext_bytes_to_decrypt)} bytes) dan kunci ({len(self.current_active_key_bytes)} bytes) harus sama.")
            self._update_gui_status("Dekripsi gagal: Panjang ciphertext dan kunci tidak cocok.")
            return
        try:
            decrypted_result_bytes = xor_bytes_operation(ciphertext_bytes_to_decrypt, self.current_active_key_bytes)
        except ValueError as e:
            messagebox.showerror("Kesalahan Operasi", f"Terjadi kesalahan saat dekripsi: {e}")
            self._update_gui_status(f"Dekripsi gagal: {e}")
            return
        self.result_output_text.config(state=tk.NORMAL)
        self.result_output_text.delete("1.0", tk.END)
        try:
            decrypted_plaintext_string = decrypted_result_bytes.decode('utf-8')
            self.result_output_text.insert(tk.END, decrypted_plaintext_string)
            self._update_gui_status("Pesan berhasil didekripsi (hasil ditampilkan sebagai teks UTF-8).")
        except UnicodeDecodeError:
            self.result_output_text.insert(tk.END, f"Data Biner (Hex): {decrypted_result_bytes.hex()}")
            self._update_gui_status("Pesan berhasil didekripsi (hasil adalah data biner, ditampilkan sebagai Hex).")
        finally:
            self.result_output_text.config(state=tk.DISABLED)

print("Definisi fungsi dan kelas OTPApplicationGUI siap.")

Definisi fungsi dan kelas OTPApplicationGUI siap.


In [ ]:
# ================================= SEL 2: EKSEKUSI =================================
# (Pastikan Sel 1 sudah dijalankan sebelumnya)

# Cek apakah kelas sudah terdefinisi (opsional, untuk debugging)
try:
    OTPApplicationGUI
    print("Kelas OTPApplicationGUI ditemukan.")
except NameError:
    print("KESALAHAN: Kelas OTPApplicationGUI tidak ditemukan. Jalankan Sel 1 terlebih dahulu!")
    # Anda bisa menambahkan raise NameError("Jalankan sel definisi terlebih dahulu") di sini jika ingin menghentikan eksekusi.

if __name__ == '__main__': # Praktik baik, meskipun di notebook fungsinya sedikit berbeda
    print("--------------------------------------------------------------------")
    print("PERHATIAN:")
    print("Sebuah jendela GUI program OTP akan segera muncul secara terpisah.")
    print("Sel Jupyter Notebook ini akan menampilkan status 'berjalan' (In [*]:).")
    print("Ini NORMAL dan berarti jendela GUI sedang aktif.")
    print("Untuk MENGHENTIKAN dan agar sel ini selesai, Anda harus MENUTUP JENDELA GUI tersebut (klik tombol 'X').")
    print("--------------------------------------------------------------------")
    
    # Membuat window utama aplikasi
    main_application_root = tk.Tk()
    
    # Membuat instance dari aplikasi OTP kita
    otp_app_instance = OTPApplicationGUI(main_application_root)
    
    # Memulai event loop Tkinter
    # Baris ini akan 'memblokir' eksekusi sel ini sampai jendela GUI ditutup.
    main_application_root.mainloop() 
    
    print("--------------------------------------------------------------------")
    print("Jendela GUI program OTP telah ditutup.")
    print("Eksekusi sel ini telah selesai.")
    print("--------------------------------------------------------------------")

Kelas OTPApplicationGUI ditemukan.
--------------------------------------------------------------------
PERHATIAN:
Sebuah jendela GUI program OTP akan segera muncul secara terpisah.
Sel Jupyter Notebook ini akan menampilkan status 'berjalan' (In [*]:).
Ini NORMAL dan berarti jendela GUI sedang aktif.
Untuk MENGHENTIKAN dan agar sel ini selesai, Anda harus MENUTUP JENDELA GUI tersebut (klik tombol 'X').
--------------------------------------------------------------------
